In [1]:
import helper_functions

In [2]:
from scipy.spatial import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn import preprocessing
from matplotlib.ticker import MaxNLocator

In [3]:
def extract_connectivity_mat(dataset_key):
    dat = pd.read_csv('../Data/'+dataset_key+'.csv')
    dat = dat.drop(columns=['other'])
    unnamed_cols = list(dat.columns[np.where(np.array(['Unnamed' in col for col in dat.columns]) == True)[0]])
    dat = dat.drop(columns=unnamed_cols)
    dat = dat.rename(columns={"vc3": "vc3l", 'vm7':'vm7d', '1':'vm7v'})
    dat = dat.sort_index(axis='columns')
    dat = dat.replace(np.nan, int(0))
    dat = dat > 0
    dat = dat.astype(int)

    return dat

In [4]:
W_Ellis_Mel_F = extract_connectivity_mat('Ellis_DMel_female')
W_Ellis_Mel_M = extract_connectivity_mat('Ellis_DMel_male')
W_Ellis_Sech = extract_connectivity_mat('Ellis_DSec_female')
W_Ellis_Sim = extract_connectivity_mat('Ellis_DSim_female')

In [5]:
# Compute base connection probabilities
def conn_probs(W):
    # needs kcxpn
    acp = np.mean(W>0,0)
    acp = acp / np.sum(acp)
    return acp

In [6]:
acp_sech = conn_probs(W_Ellis_Sech)

In [7]:
# for 1000 shuffles, compute acp
num_shuffles = 1000
all_acp_mel_f = []
all_acp_sim = []
all_acp_mel_m = []
for iter in range(num_shuffles):
    W_shuff_mel_f = helper_functions.fixed_pn_fixed_kc(W_Ellis_Mel_F.T)
    W_shuff_mel_m = helper_functions.fixed_pn_fixed_kc(W_Ellis_Mel_M.T)
    W_shuff_sim = helper_functions.fixed_pn_fixed_kc(W_Ellis_Sim.T)

    all_acp_mel_f.append(conn_probs(W_shuff_mel_f.T))
    all_acp_mel_m.append(conn_probs(W_shuff_mel_m.T))
    all_acp_sim.append(conn_probs(W_shuff_sim.T))


In [8]:
# Compute p-values
num_gloms = W_Ellis_Sech.shape[1]
per_glom = np.zeros(num_gloms)
for iter in range(num_shuffles):
    for glom in range(num_gloms):
        if (acp_sech[glom] <= all_acp_mel_f[iter][glom]) or (acp_sech[glom] <= all_acp_sim[iter][glom]) or (acp_sech[glom] <= all_acp_mel_m[iter][glom]):
            per_glom[glom] = per_glom[glom] + 1

In [9]:
p_values = per_glom/1000

In [10]:
both_fixed = pd.DataFrame({'glomerulus': W_Ellis_Sech.columns, 'p_value': p_values})

In [28]:
both_fixed.to_csv('kc_fixed_less.csv')